In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Spark Dateframe").getOrCreate()

In [0]:
df=spark.read.option("header",True).csv('/FileStore/tables/movies.csv')
df=df.withColumnRenamed("movieId","Id")
df=df.withColumnRenamed("title","Movie_Name")
df.show()

+---+--------------------+--------------------+
| Id|          Movie_Name|              genres|
+---+--------------------+--------------------+
|  1|    Toy Story (1995)|Adventure|Animati...|
|  2|      Jumanji (1995)|Adventure|Childre...|
|  3|Grumpier Old Men ...|      Comedy|Romance|
|  4|Waiting to Exhale...|Comedy|Drama|Romance|
|  5|Father of the Bri...|              Comedy|
|  6|         Heat (1995)|Action|Crime|Thri...|
|  7|      Sabrina (1995)|      Comedy|Romance|
|  8| Tom and Huck (1995)|  Adventure|Children|
|  9| Sudden Death (1995)|              Action|
| 10|    GoldenEye (1995)|Action|Adventure|...|
| 11|American Presiden...|Comedy|Drama|Romance|
| 12|Dracula: Dead and...|       Comedy|Horror|
| 13|        Balto (1995)|Adventure|Animati...|
| 14|        Nixon (1995)|               Drama|
| 15|Cutthroat Island ...|Action|Adventure|...|
| 16|       Casino (1995)|         Crime|Drama|
| 17|Sense and Sensibi...|       Drama|Romance|
| 18|   Four Rooms (1995)|              

In [0]:
df2=spark.read.option("header",True).csv('/FileStore/tables/ratings.csv')
df2=df2.withColumnRenamed("movieId","Movie_ID")
df2.show()

+------+--------+------+----------+
|userId|Movie_ID|rating| timestamp|
+------+--------+------+----------+
|     1|      31|   2.5|1260759144|
|     1|    1029|   3.0|1260759179|
|     1|    1061|   3.0|1260759182|
|     1|    1129|   2.0|1260759185|
|     1|    1172|   4.0|1260759205|
|     1|    1263|   2.0|1260759151|
|     1|    1287|   2.0|1260759187|
|     1|    1293|   2.0|1260759148|
|     1|    1339|   3.5|1260759125|
|     1|    1343|   2.0|1260759131|
|     1|    1371|   2.5|1260759135|
|     1|    1405|   1.0|1260759203|
|     1|    1953|   4.0|1260759191|
|     1|    2105|   4.0|1260759139|
|     1|    2150|   3.0|1260759194|
|     1|    2193|   2.0|1260759198|
|     1|    2294|   2.0|1260759108|
|     1|    2455|   2.5|1260759113|
|     1|    2968|   1.0|1260759200|
|     1|    3671|   3.0|1260759117|
+------+--------+------+----------+
only showing top 20 rows



In [0]:
from pyspark.sql import functions as F

In [0]:
df3= df2.groupBy('Movie_ID').agg(F.count(df2.rating).alias("Total_No_Of_Rating"), F.avg(df2.rating).alias("Average_Rating"))

In [0]:
df4 = df3.join(df,df3["Movie_ID"]==df["Id"])
df4.show()

+--------+------------------+------------------+------+--------------------+--------------------+
|Movie_ID|Total_No_Of_Rating|    Average_Rating|    Id|          Movie_Name|              genres|
+--------+------------------+------------------+------+--------------------+--------------------+
|    2294|                53|3.2735849056603774|  2294|         Antz (1998)|Adventure|Animati...|
|     296|               324| 4.256172839506172|   296| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|    3210|                52|3.6153846153846154|  3210|Fast Times at Rid...|Comedy|Drama|Romance|
|   88140|                22| 3.659090909090909| 88140|Captain America: ...|Action|Adventure|...|
|  115713|                26|3.9423076923076925|115713|   Ex Machina (2015)|Drama|Sci-Fi|Thri...|
|   27317|                 6|              3.75| 27317|Audition (Ôdishon...|Drama|Horror|Myst...|
|    1090|                68|3.9338235294117645|  1090|      Platoon (1986)|           Drama|War|
|    2088|          

In [0]:
df5 = df4.drop("Id","genres")
df5.show()

+--------+------------------+------------------+--------------------+
|Movie_ID|Total_No_Of_Rating|    Average_Rating|          Movie_Name|
+--------+------------------+------------------+--------------------+
|    2294|                53|3.2735849056603774|         Antz (1998)|
|     296|               324| 4.256172839506172| Pulp Fiction (1994)|
|    3210|                52|3.6153846153846154|Fast Times at Rid...|
|   88140|                22| 3.659090909090909|Captain America: ...|
|  115713|                26|3.9423076923076925|   Ex Machina (2015)|
|   27317|                 6|              3.75|Audition (Ôdishon...|
|    1090|                68|3.9338235294117645|      Platoon (1986)|
|    2088|                25|              2.54|       Popeye (1980)|
|    2136|                18|2.6944444444444446|Nutty Professor, ...|
|    6240|                 2|               3.0| One Good Cop (1991)|
|    3959|                16|            3.4375|Time Machine, The...|
|    3606|          

In [0]:
df6= df5.where(df5["Total_No_Of_Rating"]>=10)

In [0]:
df7=df6.orderBy(df6.Average_Rating.desc()).limit(20)
df7.show()

+--------+------------------+-----------------+--------------------+
|Movie_ID|Total_No_Of_Rating|   Average_Rating|          Movie_Name|
+--------+------------------+-----------------+--------------------+
|    1939|                11|4.636363636363637|Best Years of Our...|
|    3469|                12|4.541666666666667|Inherit the Wind ...|
|     858|               200|           4.4875|Godfather, The (1...|
|     318|               311|4.487138263665595|Shawshank Redempt...|
|    1948|                12|4.458333333333333|    Tom Jones (1963)|
|    8132|                11|4.454545454545454|    Gladiator (1992)|
|    1945|                29|4.448275862068965|On the Waterfront...|
|    1147|                16|           4.4375|When We Were King...|
|     926|                38|4.434210526315789|All About Eve (1950)|
|    1217|                26|4.423076923076923|          Ran (1985)|
|     969|                50|             4.42|African Queen, Th...|
|    3035|                17|4.411

In [0]:
df7.write.parquet("/FileStore/tables/output4.parquet")

df_parquet=spark.read.parquet("/FileStore/tables/output4.parquet")

#print(type(df_parquet))

#df_parquet.show()

df_parquet.write.json("/FileStore/tables/outJson.json")

In [0]:
json_read=spark.read.json("/FileStore/tables/outJson.json")
json_read.show()

+-----------------+--------+--------------------+------------------+
|   Average_Rating|Movie_ID|          Movie_Name|Total_No_Of_Rating|
+-----------------+--------+--------------------+------------------+
|4.636363636363637|    1939|Best Years of Our...|                11|
|4.541666666666667|    3469|Inherit the Wind ...|                12|
|           4.4875|     858|Godfather, The (1...|               200|
|4.487138263665595|     318|Shawshank Redempt...|               311|
|4.458333333333333|    1948|    Tom Jones (1963)|                12|
|4.454545454545454|    8132|    Gladiator (1992)|                11|
|4.448275862068965|    1945|On the Waterfront...|                29|
|           4.4375|    1147|When We Were King...|                16|
|4.434210526315789|     926|All About Eve (1950)|                38|
|4.423076923076923|    1217|          Ran (1985)|                26|
|             4.42|     969|African Queen, Th...|                50|
|4.411764705882353|    3035|Mister